In [ ]:
import numpy as np 
import pandas as pd
import re 
import os

#### 

This sheet is for the consolidated data for the month of Aug - Oct for the files that start with the initials of 38.
The script was run for the files that started with the file 31 and this was left.



In [2]:
df = pd.read_csv(r"C:\Users\S1130866\Desktop\Product_Automate\product.csv")
products = [i for i in set(df.Products.str.lower().tolist()) if not isinstance(i, (int, float)) and i != '0']


In [3]:
## LOading the import files 

In [4]:
df1 = pd.read_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\File to be worked on\38\Import\38_Imp_TY2_Aug-22.xlsx")
df2 = pd.read_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\File to be worked on\38\Import\38_Imp_TY2_Sep-22.xlsx")
df3 = pd.read_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\File to be worked on\38\Import\38_Imp_TY2_Oct-22.xlsx")


In [5]:
print(df1.shape)
print(df2.shape)
print(df3.shape)

(56233, 25)
(56301, 25)
(51918, 25)


In [6]:
# Loading the export files 

In [7]:
df4 = pd.read_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\File to be worked on\38\Export\38_Exp_TY2_Aug-22.xlsx")
df5 = pd.read_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\File to be worked on\38\Export\38_Exp_TY2_Sep-22.xlsx")
df6 = pd.read_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\File to be worked on\38\Export\38_Exp_TY2_Oct-22.xlsx")


In [8]:
print(df4.shape)
print(df5.shape)
print(df6.shape)

(26131, 22)
(27751, 22)
(23914, 22)


In [9]:
## Merging the datasets and copiling them into a single one ........

In [10]:
## Import 


df_imp = pd.concat([df1,df2,df3],ignore_index=True)
df_exp = pd.concat([df4,df5,df6],ignore_index=True)

In [11]:
df_imp.shape

(164452, 25)

In [12]:
df_exp.shape

(77796, 22)

----------------------- RUNNING THE SCRIPTS ON THESE DATASETS -----------------

In [13]:
map_24d = { v: '2 4 d' for v in ['2 4-d', '24 d', '2,4-d', '2 4 d', '2 4- d', '2 4 -d', '2 4d',
       '2, 4 d', '2, 4-d', '2,4 - d', '2,4 d', '2,4- d', '2,4d', '2-4-d',
       '24d', '2-4 d', '24- d', '2-4- d', '2-4d', '24-d', '2, 4- d',
       '2, 4d', '2- 4-d']}


def find_prod(x):
   
    # a = ('azoxystrobin')
    if re.search('R&D|RESEARCH|FREE SAMPLE',x) :
        return ""
    temp_products = list(set([i for i in products if i in re.split(' |\(|\)|\+|\\|/|\*|-|\s|\d',x.lower())]))
    if re.search('[2][-,\s]*[4][-\s]*[d]',x) :
        temp_products += [map_24d[re.search('[2][-,\s]*[4][-\s]*[d]',x)[0]]]
    #if re.search('AZOXY.{,50}|azoxy.{,50}',x):
    #   temp_products += a
    if re.search('POTASH|potash|NPS|nps|PHOSPHATE|phosphate|SULFATE|sulfate|SULPHATE|sulphate',x):
        if len(temp_products)<1:
            temp_products += 'fertilizer'
    if len(temp_products)>1 :
        temp_products = [i for i in temp_products if i!='pesticide']
        temp_products = [i for i in temp_products if i!='pesticides']
    return temp_products

  

In [14]:
temp = df_imp['Goods Description'].apply(lambda x: find_prod(x))
# temp = temp.apply(lambda x:pesticide_check(x))

temp_processed = temp.apply(lambda x: ", ".join(x) if len(x)>=1 else "")

In [15]:
temp_processed.replace('a, z, o, x, y, s, t, r, o, b, i, n','azoxystrobin',inplace = True)
temp_processed.replace('f, e, r, t, i, l, i, z, e, r','fertilizer',inplace = True)
temp_processed.replace('','Others',inplace = True)

In [16]:
df_imp['Product'] = temp_processed

In [17]:
df_imp['Product'].value_counts()

Others             147582
oil                  2425
calcium              1693
control              1349
total                1166
                    ...  
fluxametamide           1
ready, master           1
ready, calcium          1
control, sodium         1
methoprene              1
Name: Product, Length: 456, dtype: int64

In [18]:
def suf(x):
    temp_suffix = [] 
    if re.search('tech.{,50}|TECH.{,50}',x):
        temp_suffix = "TECH"
    elif re.search('IPA SALT',x):
        temp_suffix = "IPA SALT"
    elif re.search('62% IPA SALT',x):
        temp_suffix = "62% IPA SALT"
    elif re.search('41% SL',x):
        temp_suffix = '41% SL'
    elif re.search('ALION PLUS.{50}',x):
        temp_suffix = "ALION PLUS"
    else :
        temp_suffix = "Others"
    return temp_suffix

In [19]:
df_imp['suffix'] =  df_imp['Goods Description'].apply(lambda x: suf(x))

In [20]:
#Writing the concerned file
df_imp.to_csv('38_Imp_TY2_compiled-22.csv')

------ For the export data ----------

In [21]:
temp = df_exp['Goods Description'].apply(lambda x: find_prod(x))
# temp = temp.apply(lambda x:pesticide_check(x))

temp_processed = temp.apply(lambda x: ", ".join(x) if len(x)>=1 else "")

#temp_processed.replace('a, z, o, x, y, s, t, r, o, b, i, n','azoxystrobin',inplace = True)
temp_processed.replace('f, e, r, t, i, l, i, z, e, r','fertilizer',inplace = True)
temp_processed.replace('','Others',inplace = True)
df_exp['Product'] = temp_processed
df_exp['suffix'] =  df_exp['Goods Description'].apply(lambda x: suf(x))

#Writing the concerned file
df_exp.to_csv('38_Exp_TY2_compiled-22.csv')

In [22]:
pwd


'C:\\Users\\S1130866\\Desktop\\Product_Automate'